In [111]:
import json
import time
import pickle

import requests
import pandas as pd

url_main = 'https://api.hh.ru/vacancies'
param = "?"
hosts = ["host=hh.ua", "host=hh.ru", "host=career.ru", "host=jobs.tut.by",
        "host=jobs.day.az", "host=hh.uz", "host=hh.kz", "host=headhunter.ge",
       "host=headhunter.kg"]
an = "&"
per_p = "per_page="
page = "page="
url =  'https://api.hh.ru/vacancies?host=hh.ua&per_page=100&page='

In [51]:
r = requests.get('https://api.hh.ru/vacancies?host=hh.ua&per_page=100&page=0')
print(r.status_code)
#print(r.text)
#vacancies = json.loads(r.text)
#print(json.loads(r.text)["items"][0])
#print()
#print(json.loads(r.text)["items"][0]["snippet"])
len(json.loads(r.text)["items"])

200


100

In [159]:


def load_vacancies_ids(url, hosts):
    vacancies = []
    vac_ids = []
    param = "?"
    an = "&"
    per_page = "per_page=100"
    page = "page="
    for host in hosts:
        for i in range(20):
            url_mod = url + param + host
            url_mod = url_mod + an + per_page + an + page + str(i)
            r = requests.get(url_mod)
            vac = json.loads(r.text)
            vacancies = vacancies + vac["items"]
            vac_ids = vac_ids + [i["id"] for i in vac["items"]]
    return vac_ids, vacancies



In [160]:
%%time
vac_ids, vacancies = load_vacancies_ids(url_main, hosts)

CPU times: user 8.75 s, sys: 2.24 s, total: 11 s
Wall time: 2min 6s


In [161]:
len(vac_ids)

18000

In [162]:
ids_set = set(vac_ids)

In [163]:
len(ids_set)

3653

In [164]:
list(ids_set)[100]

'28298956'

In [128]:
with open("hh_ids.dat", "wb") as ouf:
    pickle.dump(list(ids_set), ouf)

In [129]:
with open("hh_vacancies.dat", "wb") as ouf:
    pickle.dump(list(vacancies), ouf)

In [166]:

def load_vacancies_extended(url, ids):
    vacancies_ext = []
    count = 0
    for i in ids:
        url_mod = url + "/" + str(i)
        r = requests.get(url_mod)
        vacancy = json.loads(r.text)
        vacancies_ext.append(vacancy)
        count += 1
        
        if count % 500 == 0:
            print(count)
    return vacancies_ext


In [167]:
%%time
vacancies_ext = load_vacancies_extended(url_main, list(ids_set))

500
1000
1500
2000
2500
3000
3500
CPU times: user 1min 59s, sys: 13.2 s, total: 2min 13s
Wall time: 15min 52s


In [168]:
with open("hh_vacancies_ext.dat", "wb") as ouf:
    pickle.dump(list(vacancies), ouf)

In [169]:
print(vacancies_ext[10])

{'id': '29103664', 'premium': False, 'billing_type': {'id': 'standard', 'name': 'Стандарт'}, 'relations': [], 'name': 'Дизайнер', 'insider_interview': None, 'response_letter_required': False, 'area': {'id': '1002', 'name': 'Минск', 'url': 'https://api.hh.ru/areas/1002'}, 'salary': None, 'type': {'id': 'open', 'name': 'Открытая'}, 'address': {'city': 'Минск', 'street': 'пр. Дзержинского', 'building': '126', 'description': None, 'lat': 53.847395, 'lng': 27.471236, 'raw': None, 'metro': {'station_name': 'Малиновка', 'line_name': 'Московская', 'station_id': '62.472', 'line_id': '62', 'lat': 53.849722, 'lng': 27.474722}, 'metro_stations': [{'station_name': 'Малиновка', 'line_name': 'Московская', 'station_id': '62.472', 'line_id': '62', 'lat': 53.849722, 'lng': 27.474722}]}, 'allow_messages': True, 'site': {'id': 'hh', 'name': 'hh.ru'}, 'experience': {'id': 'between1And3', 'name': 'От 1 года до 3 лет'}, 'schedule': {'id': 'fullDay', 'name': 'Полный день'}, 'employment': {'id': 'full', 'name'

In [170]:

with open("hh_ids.json", "w") as ouf:
    json.dump(list(ids_set), ouf, ensure_ascii=False)
    
with open("hh_vacancies.json", "w") as ouf:
    json.dump(vacancies, ouf, ensure_ascii=False)
    
with open("hh_vacancies_ext.json", "w") as ouf:
    json.dump(vacancies_ext, ouf, ensure_ascii=False)
    

In [171]:
df = pd.DataFrame(vacancies)

In [172]:
df.head()

,address,alternate_url,apply_alternate_url,archived,area,contacts,created_at,department,employer,has_test,...,premium,published_at,relations,response_letter_required,response_url,salary,snippet,sort_point_distance,type,url
0,"{'city': 'Шклов', 'street': 'Интернациональная...",https://hh.ua/vacancy/29103667,https://hh.ua/applicant/vacancy_response?vacan...,False,"{'id': '2352', 'name': 'Шклов', 'url': 'https:...","{'name': 'Фомина Юлия', 'email': 'yuliya.fomin...",2018-11-29T19:56:43+0300,None,"{'id': '619035', 'name': 'Danone Belarus', 'ur...",False,...,False,2018-11-29T19:56:43+0300,[],False,None,None,{'requirement': 'Высшее (желательно техническо...,None,"{'id': 'open', 'name': 'Открытая'}",https://api.hh.ru/vacancies/29103667?host=hh.ua
1,None,https://hh.ua/vacancy/29103666,https://hh.ua/applicant/vacancy_response?vacan...,False,"{'id': '90', 'name': 'Томск', 'url': 'https://...",None,2018-11-29T19:56:41+0300,None,"{'id': '3637261', 'name': 'СИБ-ТОМ', 'url': 'h...",False,...,False,2018-11-29T19:56:41+0300,[],False,None,"{'from': 60000, 'to': 80000, 'currency': 'RUR'...",{'requirement': 'Умение работать в коллективе....,None,"{'id': 'open', 'name': 'Открытая'}",https://api.hh.ru/vacancies/29103666?host=hh.ua
2,"{'city': 'Киев', 'street': 'улица Казимира Мал...",https://hh.ua/vacancy/29103665,https://hh.ua/applicant/vacancy_response?vacan...,False,"{'id': '115', 'name': 'Киев', 'url': 'https://...",None,2018-11-29T19:55:59+0300,None,"{'id': '3150328', 'name': 'Qubit Labs', 'url':...",False,...,False,2018-11-29T19:55:59+0300,[],False,None,"{'from': 1500, 'to': 2500, 'currency': 'USD', ...",{'requirement': 'At least a bachelor degree in...,None,"{'id': 'open', 'name': 'Открытая'}",https://api.hh.ru/vacancies/29103665?host=hh.ua
3,"{'city': 'Минск', 'street': 'пр. Дзержинского'...",https://hh.ua/vacancy/29103664,https://hh.ua/applicant/vacancy_response?vacan...,False,"{'id': '1002', 'name': 'Минск', 'url': 'https:...",None,2018-11-29T19:55:57+0300,None,"{'id': '746617', 'name': 'ProStore', 'url': 'h...",False,...,False,2018-11-29T19:55:57+0300,[],False,None,None,{'requirement': 'Профильное высшее образование...,None,"{'id': 'open', 'name': 'Открытая'}",https://api.hh.ru/vacancies/29103664?host=hh.ua
4,"{'city': 'Смоленск', 'street': 'Чуриловский пе...",https://hh.ua/vacancy/29103661,https://hh.ua/applicant/vacancy_response?vacan...,False,"{'id': '83', 'name': 'Смоленск', 'url': 'https...",None,2018-11-29T19:55:48+0300,None,"{'id': '2688299', 'name': 'Интернет-студия Вре...",False,...,False,2018-11-29T19:55:48+0300,[],False,None,"{'from': 35000, 'to': None, 'currency': 'RUR',...",{'requirement': '1. Высшее образование ( в при...,None,"{'id': 'open', 'name': 'Открытая'}",https://api.hh.ru/vacancies/29103661?host=hh.ua


In [32]:
with open("hh_ids.dat", "rb") as inf:
    t = pickle.load(inf)

In [ ]:
with open("hh_ids.json", "wb") as ouf:
    pickle.dump(list(ids_set), ouf)